# 手动实现 K-Means 聚类算法
本 Notebook 用于学习K-Means，将一步步拆解 K-Means 算法的实现过程，帮助理解其内部机制。

### K-Means 是什么？
- K-Means（K-Means Clustering）是一种常用的无监督聚类算法。
- 它通过将数据集划分为K个集群，使得每个集群的样本相似。
- 目标：将数据集划分成K个相似的子集，每个子集内样本相似。

### 核心步骤回顾
1. 初始化聚类中心（随机数取）
2. 迭代：
    - 计算每个样本到聚类中心的距离
    - 将样本分配到距离最近的聚类中心
    - 更新聚类中心
3. 停止：当聚类中心不再改变/到最大迭代次数，停止迭代


In [1]:
import numpy as np
from sklearn.cluster import KMeans  # 用于最后对比

##  Step 1: 生成测试数据

In [2]:
# 设置随机种子以确保结果可复现
np.random.seed(42)

# 生成 100 个二维点作为测试数据
X = np.random.rand(100, 2)

print("原始数据 X shape:", X.shape)
print("前5行样本:\n", X[:5])

原始数据 X shape: (100, 2)
前5行样本:
 [[0.37454012 0.95071431]
 [0.73199394 0.59865848]
 [0.15601864 0.15599452]
 [0.05808361 0.86617615]
 [0.60111501 0.70807258]]


##  Step 2: 初始化聚类中心

In [3]:
n_clusters = 3
random_state = 42

# 设置随机种子
np.random.seed(random_state)

# 随机选择 n_clusters 个索引
indices = np.random.permutation(X.shape[0])[:n_clusters]

# 取出初始中心点
initial_centers = X[indices]

print("初始中心点 shape:", initial_centers.shape)
print("初始中心点坐标:\n", initial_centers)

初始中心点 shape: (3, 2)
初始中心点坐标:
 [[0.32078006 0.18651851]
 [0.41038292 0.75555114]
 [0.96244729 0.2517823 ]]


##  Step 3: 计算距离矩阵

In [5]:
def compute_distances(X, centers):
    n_samples, n_clusters = X.shape[0], centers.shape[0]
    distances = np.zeros((n_samples, n_clusters))
    for i in range(n_clusters):
        distances[:, i] = np.linalg.norm(X - centers[i], axis=1)
    return distances

distances = compute_distances(X, initial_centers)

print("距离矩阵 shape:", distances.shape)
print("前10行示例:\n", distances[:10])

距离矩阵 shape: (100, 3)
前10行示例:
 [[0.76608443 0.19842724 0.91331309]
 [0.58219946 0.35783928 0.41645148]
 [0.16756504 0.65128283 0.81209758]
 [0.72865899 0.36925966 1.09332222]
 [0.5921202  0.19655265 0.58203251]
 [0.83893943 0.44485107 1.18440395]
 [0.51231367 0.68790531 0.13585645]
 [0.13898999 0.61610917 0.78361135]
 [0.33864198 0.25403157 0.71256492]
 [0.1527153  0.46482238 0.53196684]]


##  Step 4: 分配簇标签

In [6]:
def assign_clusters(distances):
    return np.argmin(distances, axis=1)

labels = assign_clusters(distances)

print("簇标签 shape:", labels.shape)
print("前10个标签:\n", labels[:10])
# 可以看到argmin返回的是最小值索引

簇标签 shape: (100,)
前10个标签:
 [1 1 0 1 1 1 2 0 1 0]


##  Step 5: 更新聚类中心

In [7]:
def update_centers(X, labels, n_clusters):
    _, n_features = X.shape
    new_centers = np.zeros((n_clusters, n_features))
    for i in range(n_clusters):
        cluster_points = X[labels == i]
        if len(cluster_points) > 0:
            new_centers[i] = np.mean(cluster_points, axis=0)
    return new_centers

new_centers = update_centers(X, labels, n_clusters)

print("更新后的中心点 shape:", new_centers.shape)
print("更新后的中心点:\n", new_centers)

更新后的中心点 shape: (3, 2)
更新后的中心点:
 [[0.31979929 0.2344994 ]
 [0.41379196 0.75505474]
 [0.8492242  0.38161194]]


##  Step 6: 迭代直到收敛

In [8]:
max_iter = 300
centers = initial_centers.copy()

for iteration in range(max_iter):
    distances = compute_distances(X, centers)
    labels = assign_clusters(distances)
    new_centers = update_centers(X, labels, n_clusters)

    # 判断是否收敛
    if np.allclose(centers, new_centers):
        print(f"迭代 {iteration+1} 次后收敛")
        break

    centers = new_centers.copy()
else:
    print(f"达到最大迭代次数 {max_iter}，未完全收敛")

final_centers = centers
final_labels = assign_clusters(compute_distances(X, final_centers))

print("最终中心点:\n", final_centers)

迭代 7 次后收敛
最终中心点:
 [[0.36376248 0.20008043]
 [0.19671223 0.72161646]
 [0.81167067 0.56668218]]


## Step 7: 对比 sklearn 实现

In [9]:
sk_kmeans = KMeans(n_clusters=3, random_state=42).fit(X)
print("sklearn 中心点:\n", sk_kmeans.cluster_centers_)

sklearn 中心点:
 [[0.18520943 0.72228065]
 [0.8039633  0.57026999]
 [0.36376248 0.20008043]]
